In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/val/val_week1_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/test/test_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/clean_data_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/val/val_week1_2_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/test/test_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/clean_data_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw

## 1. Import libraries

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support

<ipython-input-2-b033c627a4bd>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


## 2. Load dataset

In [3]:
five_fold_files = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_5.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_5.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_5.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_5.csv"
    ]
}

file_validataion = {
    "week1": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/val/val_week1.csv",
    "week2": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/val/val_week1_2.csv",
    "week3": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/val/val_week1_2_3.csv",
    "week4": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/val/val_week1_2_3_4.csv"
}

file_test = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/test/test_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/test/test_week2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/test/test_week3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/test/test_week4.csv"
    ]
}

In [4]:
# Danh sách siêu tham số tốt nhất (cập nhật)
best_params = {
    "week1": {
        "ann_units_1": 64,
        "ann_dropout_1": 0.1,
        "lstm_units": 256,
        "lstm_dropout": 0.3,
        "ann_units_2": 160,
        "ann_dropout_2": 0.5,
        "learning_rate": 0.002480208992685262
    },
    "week2": {
        "ann_units_1": 32,
        "ann_dropout_1": 0.3,
        "lstm_units": 256,
        "lstm_dropout": 0.1,
        "ann_units_2": 32,
        "ann_dropout_2": 0.4,
        "learning_rate": 0.006693994895295038
    },
    "week3": {
        "ann_units_1": 32,
        "ann_dropout_1": 0.1,
        "lstm_units": 192,
        "lstm_dropout": 0.4,
        "ann_units_2": 128,
        "ann_dropout_2": 0.2,
        "learning_rate": 0.0008905259819768822
    },
    "week4": {
        "ann_units_1": 96,
        "ann_dropout_1": 0.2,
        "lstm_units": 224,
        "lstm_dropout": 0.4,
        "ann_units_2": 160,
        "ann_dropout_2": 0.5,
        "learning_rate": 0.0014711547097274132
    }
}


## Mô hình ANN-LSTM

## Dự đoán các week  với best parameters

In [5]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình ANN-LSTM với tham số truyền vào
def build_ANN_LSTM_model(params):
    model = tf.keras.Sequential()
    
    # ANN Layer 1
    model.add(layers.Dense(
        units=params.get('ann_units_1', 64),
        activation='relu',
        input_shape=(X_train.shape[1], 1)
    ))
    model.add(layers.Dropout(rate=params.get('ann_dropout_1', 0.2)))
    
    # LSTM Layer
    model.add(layers.LSTM(
        units=params.get('lstm_units', 64),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('lstm_dropout', 0.2)))
    
    # ANN Layer 2
    model.add(layers.Dense(
        units=params.get('ann_units_2', 32),
        activation='relu'
    ))
    model.add(layers.Dropout(rate=params.get('ann_dropout_2', 0.2)))
    
    # Lớp đầu ra
    model.add(layers.Dense(5, activation='softmax'))
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params.get('learning_rate', 0.001)),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
                  metrics=['accuracy'])
    return model



## 5-folds

In [6]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week]
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho ANN LSTM
        X_train = X_train.to_numpy()
        X_test = X_test.to_numpy()

        # Xây dựng mô hình với tham số tốt nhất
        model = build_ANN_LSTM_model(params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["confusion_matrices"].append(conf_matrix)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.mean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.mean(week_results["f1_score_per_label"], axis=0)
    average_confusion_matrix = np.mean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    
    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label
    })
    
    # Tạo DataFrame cho confusion matrix|
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    
    # In kết quả
    print("\n=== Average Precision, Recall, F1-Score per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)

    # Lưu kết quả tuần vào kết quả tổng quát
    week_results["average_train_times"] = average_train_time
    week_results["average_test_times"] = average_test_time
    week_results["average_metrics_df"] = metrics_df
    week_results["average_confusion_matrix"] = confusion_df
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'ann_units_1': 64, 'ann_dropout_1': 0.1, 'lstm_units': 256, 'lstm_dropout': 0.3, 'ann_units_2': 160, 'ann_dropout_2': 0.5, 'learning_rate': 0.002480208992685262}
Fold 1: Using file /kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_1.csv as test set


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 0.5972 - loss: 0.1563 - val_accuracy: 0.6283 - val_loss: 0.1409
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.6304 - loss: 0.1410 - val_accuracy: 0.6115 - val_loss: 0.1387
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6206 - loss: 0.1406 - val_accuracy: 0.6252 - val_loss: 0.1379
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.6328 - loss: 0.1397 - val_accuracy: 0.6462 - val_loss: 0.1322
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.6405 - loss: 0.1355 - val_accuracy: 0.6710 - val_loss: 0.1291
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6649 - loss: 0.1307 - val_accuracy: 0.6752 - val_loss: 0.1240
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6688 - loss: 0.1267 - val_accuracy: 0.6740 - val_loss: 0.1245
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6667 - loss: 0.1266 - val_acc

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.5872 - loss: 0.1620 - val_accuracy: 0.6115 - val_loss: 0.1402
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6263 - loss: 0.1422 - val_accuracy: 0.6317 - val_loss: 0.1386
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6338 - loss: 0.1422 - val_accuracy: 0.6451 - val_loss: 0.1340
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.6542 - loss: 0.1330 - val_accuracy: 0.6607 - val_loss: 0.1295
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6562 - loss: 0.1342 - val_accuracy: 0.6657 - val_loss: 0.1307
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6627 - loss: 0.1312 - val_accuracy: 0.6775 - val_loss: 0.1229
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6619 - loss: 0.1272 - val_accuracy: 0.6706 - val_loss: 0.1262
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.6634 - loss: 0.1289 - val_accuracy: 0.69

...

328/328 ━━━━━━━━━━━━━━━━━━━━ 22s 59ms/step - accuracy: 0.5974 - loss: 0.1640 - val_accuracy: 0.6114 - val_loss: 0.1479
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.6125 - loss: 0.1472 - val_accuracy: 0.6602 - val_loss: 0.1368
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 23s 69ms/step - accuracy: 0.6205 - loss: 0.1404 - val_accuracy: 0.6358 - val_loss: 0.1371
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.6289 - loss: 0.1389 - val_accuracy: 0.6445 - val_loss: 0.1348
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 58ms/step - accuracy: 0.6442 - loss: 0.1341 - val_accuracy: 0.6377 - val_loss: 0.1347
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 58ms/step - accuracy: 0.6321 - loss: 0.1371 - val_accuracy: 0.6823 - val_loss: 0.1294
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.6763 - loss: 0.1293 - val_accuracy: 0.6937 - val_loss: 0.1306
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 58ms/step - accuracy: 0.6879 - loss: 0.1264 - val_accurac

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 22s 60ms/step - accuracy: 0.5929 - loss: 0.1631 - val_accuracy: 0.6110 - val_loss: 0.1474
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 22s 67ms/step - accuracy: 0.6173 - loss: 0.1425 - val_accuracy: 0.6140 - val_loss: 0.1371
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - accuracy: 0.6180 - loss: 0.1400 - val_accuracy: 0.6384 - val_loss: 0.1364
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 18s 56ms/step - accuracy: 0.6264 - loss: 0.1378 - val_accuracy: 0.6674 - val_loss: 0.1340
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - accuracy: 0.6326 - loss: 0.1367 - val_accuracy: 0.6609 - val_loss: 0.1325
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 18s 56ms/step - accuracy: 0.6590 - loss: 0.1344 - val_accuracy: 0.6354 - val_loss: 0.1359
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.6670 - loss: 0.1321 - val_accuracy: 0.6937 - val_loss: 0.1256
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 18s 56ms/step - accuracy: 0.6924 - loss: 0.1267 - val_accurac

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.5925 - loss: 0.1643 - val_accuracy: 0.6114 - val_loss: 0.1396
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.6178 - loss: 0.1434 - val_accuracy: 0.6144 - val_loss: 0.1375
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 23s 70ms/step - accuracy: 0.6159 - loss: 0.1434 - val_accuracy: 0.6320 - val_loss: 0.1375
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 18s 56ms/step - accuracy: 0.6389 - loss: 0.1363 - val_accuracy: 0.6133 - val_loss: 0.1355
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.6466 - loss: 0.1395 - val_accuracy: 0.6659 - val_loss: 0.1289
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 18s 56ms/step - accuracy: 0.6684 - loss: 0.1335 - val_accuracy: 0.6972 - val_loss: 0.1238
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.6834 - loss: 0.1300 - val_accuracy: 0.7086 - val_loss: 0.1207
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 57ms/step - accuracy: 0.6915 - loss: 0.1296 - val_accurac

## Kết quả 5 folds

In [7]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_confusion_matrix = week_result["average_confusion_matrix"]

    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print("\nAverage Precision, Recall, F1-Score per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Train Time: 429.6909 seconds
Average Test Time: 1.1380 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.710428        0.716333          0.709224
1      1           0.504127        0.102429          0.166560
2      2           0.673780        0.454937          0.538116
3      3           0.629774        0.500599          0.554668
4      4           0.842180        0.920544          0.879063

Average Confusion Matrix:
       0    1     2     3       4
0  429.8  4.6  11.4   8.6   145.6
1   39.8  9.0   3.4   3.6    31.8
2   38.0  0.2  74.8   6.8    44.6
3   18.8  1.4   4.4  83.6    58.8
4   78.2  1.0  17.6  30.6  1476.0

=== Results for week2 ===
Average Train Time: 659.0878 seconds
Average Test Time: 1.5735 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.620921        0.375667

## Kết quả trên tập test

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình ANN-LSTM với tham số truyền vào
def build_ANN_LSTM_model(params):
    model = tf.keras.Sequential()
    
    # ANN Layer 1
    model.add(layers.Dense(
        units=params.get('ann_units_1', 64),
        activation='relu',
        input_shape=(X_train.shape[1], 1)
    ))
    model.add(layers.Dropout(rate=params.get('ann_dropout_1', 0.2)))
    
    # LSTM Layer
    model.add(layers.LSTM(
        units=params.get('lstm_units', 64),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('lstm_dropout', 0.2)))
    
    # ANN Layer 2
    model.add(layers.Dense(
        units=params.get('ann_units_2', 32),
        activation='relu'
    ))
    model.add(layers.Dropout(rate=params.get('ann_dropout_2', 0.2)))
    
    # Lớp đầu ra
    model.add(layers.Dense(5, activation='softmax'))
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params.get('learning_rate', 0.001)),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
                  metrics=['accuracy'])
    return model



In [9]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"]
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy()
    X_test = X_test.to_numpy()

    # Xây dựng mô hình với tham số tốt nhất
    model = build_ANN_LSTM_model(params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    
    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix
    })
    
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [10]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 30s 25ms/step - accuracy: 0.3119 - loss: 0.2424 - val_accuracy: 0.0384 - val_loss: 0.3108
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 28s 24ms/step - accuracy: 0.3960 - loss: 0.2142 - val_accuracy: 0.0881 - val_loss: 0.3048
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 27s 24ms/step - accuracy: 0.4994 - loss: 0.1839 - val_accuracy: 0.0950 - val_loss: 0.3129
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 28s 25ms/step - accuracy: 0.5331 - loss: 0.1689 - val_accuracy: 0.1938 - val_loss: 0.2692
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 32s 28ms/step - accuracy: 0.5467 - loss: 0.1623 - val_accuracy: 0.4006 - val_loss: 0.2337
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 37s 24ms/step - accuracy: 0.5785 - loss: 0.1470 - val_accuracy: 0.6361 - val_loss: 0.1845
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 28s 25ms/step - accuracy: 0.6030 - loss: 0.1362 - val_accuracy: 0.6987 - val_loss: 0.1532
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 28s 25ms/step - accuracy: 0.6117 - loss: 0.13

In [11]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 43s 36ms/step - accuracy: 0.2214 - loss: 0.2528 - val_accuracy: 0.0000e+00 - val_loss: 0.4198
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 40s 35ms/step - accuracy: 0.2229 - loss: 0.2506 - val_accuracy: 0.0000e+00 - val_loss: 0.4202
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.2193 - loss: 0.2504 - val_accuracy: 0.0000e+00 - val_loss: 0.4089
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 44s 39ms/step - accuracy: 0.2186 - loss: 0.2504 - val_accuracy: 0.0000e+00 - val_loss: 0.4244
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 40s 36ms/step - accuracy: 0.2202 - loss: 0.2508 - val_accuracy: 0.0000e+00 - val_loss: 0.4214
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.2216 - loss: 0.2506 - val_accuracy: 0.0000e+00 - val_loss: 0.4140
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 46s 41ms/step - accuracy: 0.2238 - loss: 0.2510 - val_accuracy: 0.0000e+00 - val_loss: 0.3980
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 41s 37ms/step - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 43s 34ms/step - accuracy: 0.3201 - loss: 0.2374 - val_accuracy: 4.9888e-04 - val_loss: 0.3764
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 38s 34ms/step - accuracy: 0.4372 - loss: 0.2035 - val_accuracy: 0.1532 - val_loss: 0.3070
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 38s 34ms/step - accuracy: 0.4930 - loss: 0.1817 - val_accuracy: 0.1938 - val_loss: 0.2718
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 43s 38ms/step - accuracy: 0.5532 - loss: 0.1590 - val_accuracy: 0.3894 - val_loss: 0.2231
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 38s 34ms/step - accuracy: 0.6262 - loss: 0.1343 - val_accuracy: 0.4667 - val_loss: 0.1881
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 38s 34ms/step - accuracy: 0.6652 - loss: 0.1189 - val_accuracy: 0.6273 - val_loss: 0.1810
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 40s 35ms/step - accuracy: 0.6939 - loss: 0.1080 - val_accuracy: 0.6623 - val_loss: 0.1491
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 42s 37ms/step - accuracy: 0.7248 - loss: 

In [13]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 61s 52ms/step - accuracy: 0.3158 - loss: 0.2397 - val_accuracy: 0.0000e+00 - val_loss: 0.3860
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 58s 52ms/step - accuracy: 0.4096 - loss: 0.2143 - val_accuracy: 0.0469 - val_loss: 0.3409
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 60s 53ms/step - accuracy: 0.4686 - loss: 0.1900 - val_accuracy: 0.2869 - val_loss: 0.2462
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 59s 52ms/step - accuracy: 0.5047 - loss: 0.1752 - val_accuracy: 0.3165 - val_loss: 0.2726
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 66s 58ms/step - accuracy: 0.5318 - loss: 0.1634 - val_accuracy: 0.3290 - val_loss: 0.2479
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 59s 52ms/step - accuracy: 0.5566 - loss: 0.1533 - val_accuracy: 0.2647 - val_loss: 0.2766
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 58s 52ms/step - accuracy: 0.5796 - loss: 0.1464 - val_accuracy: 0.3143 - val_loss: 0.2226
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 60s 53ms/step - accuracy: 0.5970 - loss: 

In [14]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 1460.43 seconds
  Test Time: 0.91 seconds
  Precision: [0.72463768 0.22222222 0.48598131 0.49193548 0.89901207]
  Recall: [0.66666667 0.62962963 0.50485437 0.58095238 0.81655035]
  F1-Score: [0.69444444 0.32850242 0.4952381  0.53275109 0.85579937]
  Confusion Matrix:
[[250  55  18   8  44]
 [  9  34   1   1   9]
 [ 22  10  52   3  16]
 [  4   9   8  61  23]
 [ 60  45  28  51 819]]
Week 2:
  Train Time: 2168.18 seconds
  Test Time: 1.12 seconds
  Precision: [0.         0.         0.         0.         0.61158537]
  Recall: [0. 0. 0. 0. 1.]
  F1-Score: [0.       0.       0.       0.       0.758986]
  Confusion Matrix:
[[   0    0    0    0  375]
 [   0    0    0    0   54]
 [   0    0    0    0  103]
 [   0    0    0    0  105]
 [   0    0    0    0 1003]]
Week 3:
  Train Time: 1919.53 seconds
  Test Time: 1.21 seconds
  Precision: [0.69871795 0.20138889 0.4025974  0.38961039 0.91780822]
  Recall: [0.58133333 0.53703704 0.60194